<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js098_electronsTDKS1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [6]:
#@title js098_electronsTDKS1D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js098_electronsTDKS1D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js098_electronsTDKS1D
//    Copyright(C) 2018-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.03.25 created, last updated on 2018.12.04
//    ver 0.0.1  2019.01.26 v1, last updated on 2021.09.13
//    ver 0.0.2  2021.11.06 v2, last updated on 2021.11.06
//    ver 0.0.3  2023.05.08 v3, last updated on 2023.09.09
//
// -------------------- time dependent Kohn-Sham 1D : LDA + spin
//
//  time dependent Kohn-Sham 1D : LDA + spin
//
// - solve time dependent Kohn-Sham equation : approximation
//      N. Watanabe and M. Tsukada: Phys. Rev., E65, 036705 (2002)
// - Vxc: LDA
//      J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981)
//
//  many electron system --> one electron approximation
//    one electron ,  others -> consider as effective potential Veff
//    Kohn-Sham equation {(-d^2/dr^2)+Veff(r)} |i> = e_i |i>,  Veff(r)=Vext+VH+Vx+Vc
//      Veff: one electron feels effective potential
//      Vext: external Potential
//      Vh  : Hartree potential (electron charge -> electro-static potential)
//      Vx,Vc :electron exchange and correlation potential (LDA:Perdew-Zunger)
//    successive approximation
//      solve Kohn-Sham -> electron state,occupation -> electron charge density -> Veff
//
//   quantum electron dynamics
//   - real-space : sttate psi(i) = ph(i)*si (si = spinUp or spinDown)
//   - time dependent Kohn-Sham equation :
//      N. Watanabe and M. Tsukada: Physical Review E, Vol 65, No 3, 036705 (2002)
//      (arXiv physics/0112015)
//      H ph(ri,t) = i d ph(ri,t)/dt, H = -D2/2 + Veff
//      ph(ri,t+dt) = exp(-idt*H) ph(ri,t)
//      approximation :
//      ph(ri,t+dt) = exp(idt/2*D/2)exp(-idt*Veff)exp(idt/2*D/2)ph(ri,t)
//      Veff: one electron feels effective potential
//        Veff = Vext + Vh + Vx + Vc
//          Vx + Vc : LDA( J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981) )
//          Vext: external Potential
//          Vh  : Hartree potential (electron charge -> electro-static potential)
//          Vx,Vc :electron exchange and correlation potential (LDA:Perdew-Zunger)
//  loss process
//      ph(i) steepest descent method: |ph(next)> = |ph(i)> - dump{H-Ei}|ph(i)>
//      spin : as another degree of freedom of state : psi(i) = ph(i)*si
//      <psi(i)|psi(j)> = 0 (si,sj parallel), or = 0 (<ph(i)|ph(j)> = 0)
//      {psi(0),..,psi(N)} Gram-Schmidt orthogonallization
//
//  procedure : evolve time dependent Kohn-Sham equation
//      ( state orthogonallity conserved in this process )
//
//   (1) set initial orbit
//      set external potential Vext
//      set initial orbit ph(i)
//
//   (2) evolve time dt/2
//      t <-- t + dt/2
//
//   (3) evolve kinetic part dt/2 - exp(idt/2*D/2)*
//      krStep(ph(i),0.5*dt);
//
//   (4) set electron density
//      rho <-- sum(|ph(i)|^2,i)
//
//   (5) set effective potential
//      Veff = Vext + Vh + Vx + Vc
//      Vh <-- rho (Poisson eq. ,SOR iteration)
//      Vx,Vc <-- rho (LDA:Perdew-Zunger)
//
//   (6) evolve potential part dt - exp(-idt*Veff)*
//      phaseStep(ph(i), Veff, dt);
//
//   (7) evolve time dt/2
//      t <-- t + dt/2
//
//   (8) evolve kinetic part dt/2 - exp(idt/2*D/2)*
//      krStep(ph(i,t+dt/2),0.5*dt);
//
//   goto (2)
//
// --------------------
*/

const electronsTDKS1D = (function(){ // ====================  electronsTDKS1D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;			// (m) 1(au) = g_auLength (m)
	const g_auTime = 2.418884326e-17;			// (s) 1(au) = g_auTime (s)
	const g_auEnergy = 4.35974465e-18;			// (J) 1(au) = g_auEnergy (J)
	const g_au2eV = 27.211386;					// (eV) 1(au) = 27.211386 (eV)
	const g_nOrbitMax = 10;						// maximum number of orbit
	const g_nStateMax = g_nOrbitMax*2;			// maximum number of state |orbit>,spin
	const g_nxMax = 640;						// maximum number of g_NNx

	let g_iterCount = 0;						// iteration count
	let g_sysTime = 0.0;						// (au) system time
	let g_dx = 1.0/4.0;							// (au) x-division
	let g_timeStep = 0.5*g_dx*g_dx;				// (au) time step dt
	let g_lylz = 16.0*16.0;						// imaginary spreading in the y,z direction
	let g_NNx = 320;							// number of space x-division
	let g_numberOfOrbit = 6;					// number of orbit
	let g_numberOfState = g_numberOfOrbit*2;	// number of state |orbit>,spin
	let g_numberOfElectron = 2.0;				// number of electron

	let g_qmdSW = 0;							// 0-OFF 1-qmd ON (move nuclear)
	let g_lossSW = 0;							// 0-lossOFF 1-lossON 2-loss+trans 3-loss+trans,t-stop
	let g_dampingFactor = 0.01;					// damping factor in setLoss()

	const g_psis = dimInt2( g_nOrbitMax, 2 );	// 0-orbit index(0..g_numberOfOrbit-1), 1-spin(1,-1)
	const g_occ = dim1( g_nStateMax );			// occupation of state 0.0 ... 1.0
	const g_energy = dim1( g_nStateMax );		// state energy g_energy[istate]
	const g_psi = dim3( g_nOrbitMax, g_nxMax, 2 ); // wave function g_psi[orbit][ix][0 or 1] 0:Re, 1:Im
	const g_wrk = dim2( g_nxMax, 2 );			// work orbit in teepestDescent() g_wrk[ix][0 or 1]
	const g_vv = dim1( g_nxMax );				// effective potential g_vv[ix]
	const g_vvext = dim1( g_nxMax );			// external potential g_vvext[ix]
	const g_vvh = dim1( g_nxMax );				// Hartree (electro-static) potential caused by g_rho g_vvh[ix]
	const g_vvx = dim1( g_nxMax );				// exchange potenial g_vvx[ix]
	const g_vvc = dim1( g_nxMax );				// correlation potential g_vvc[ix]
	const g_rho = dim1( g_nxMax );				// electron charge density g_rho[ix]

	const g_bRe = dim1( g_nxMax );				// work b vector in kxStep()
	const g_bIm = dim1( g_nxMax );				// work b vector in kxStep()
	const g_uRe = dim1( g_nxMax );				// work u vector in kxStep()
	const g_uIm = dim1( g_nxMax );				// work u vector in kxStep()

	// for QMD
	const g_NNN = 8;							// nuc array max
	const g_jelliumRadius = 1.0;				// nuc potential jellium radius

	let g_nucMax = 2;							// maximum number of nucleus
	const g_nucMass = dim1( g_NNN );				// mass of nucleus
	const g_nucCharge = dim1( g_NNN );			// charge of nucleus
	const g_nucxx = dim1( g_NNN );				// (au) nuclear x-position
	const g_nucvx = dim1( g_NNN );				// (m/s) nuclear x-velocity
	const g_nucfx = dim1( g_NNN );				// (N) nuclear x-force

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a=[];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		const vIndex = 0;

		g_iterCount = 0;
		g_sysTime = 0.0;
		setInitialState(theme);
		setExternalPotential(vIndex);
	}

	function setInitialState(phIndex) {
		const xMax=g_NNx*g_dx;

		if (phIndex==0) { // 2-electron up,up
			g_numberOfOrbit = 2;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  2.0;
			//setGaussianWaveAndSpin(iOrbit, xPos, width, momentum, spin) spin 1:up -1:down
			setGaussianWaveAndSpin(0, xMax*0.25, 1.6, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(1, xMax*0.5,  1.6, 0.0,  1); // 1:up
		} else if (phIndex==1) { // 2-electron up,down
			g_numberOfOrbit = 2;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  2.0;
			setGaussianWaveAndSpin(0, xMax*0.25, 1.6, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(1, xMax*0.5,  1.6, 0.0, -1); //-1:down
		} else if (phIndex==2) { // 3-electron up,up,down
			g_numberOfOrbit = 3;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  3.0;
			setGaussianWaveAndSpin(0, xMax*0.25, 1.6, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(1, xMax*0.5,  1.6, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(2, xMax*0.65, 1.6, 0.0, -1); // 1:down
		} else if (phIndex==3) { // 4-electron up,up,up,up
			g_numberOfOrbit = 4;
			g_numberOfState = g_numberOfOrbit*2;
			g_numberOfElectron =  4.0;
			setGaussianWaveAndSpin(0, xMax*0.25, 1.6, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(1, xMax*0.5,  1.6, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(2, xMax*0.65, 1.6, 0.0,  1); // 1:up
			setGaussianWaveAndSpin(3, xMax*0.5,  1.6, 2.0,  1); // 1:up
		}
	}

	function setExternalPotential(vIndex) {
		const xMax=g_NNx*g_dx;

		if (vIndex==0) {
			const xPos = 0.5*xMax;
			const vvextAtXMax = 8.0;
			setHarmonicPotential(xPos, vvextAtXMax)
		}
	}

	function setGaussianWaveAndSpin(iOrbit, xPos, width, momentum, spin) {
		setGaussianWave(g_psi[iOrbit], xPos, width, momentum);
		setElectronStatesAndOccupation(iOrbit, spin);
	}

	function setElectronStatesAndOccupation(iOrbit, spin) {

		// set electron state: orbit -> g_psis[state][0], spin -> g_psis[state][1], and g_occ[]
		const iState = iOrbit*2;
		g_psis[iState][0] = iOrbit; g_psis[iState][1] = 1;
		g_psis[iState+1][0] = iOrbit; g_psis[iState+1][1] = -1;
		if (spin==1) { // 1-up spin
			g_occ[iState] = 1.0;
			g_occ[iState+1] = 0.0;
		} else if (spin==-1) { // -1-down spin
			g_occ[iState] = 0.0;
			g_occ[iState+1] = 1.0;
		} else if (spin==2) { // 2-up and down spin
			g_occ[iState] = 1.0;
			g_occ[iState+1] = 1.0;
		}
	}

	function setGaussianWave(ph, xPos, width, momentum) { // ph[][]
		const nnx=g_NNx;

		const a = Math.pow(2.0*Math.PI*width*width,-0.25);
		for (let i=1; i<nnx-1; i++) {
			const x = i*g_dx;
			const phAb = a*Math.exp(-((x-xPos)/(2.0*width))*((x-xPos)/(2.0*width)));
			const phPh = momentum*x;
			ph[i][0] = phAb*Math.cos(phPh);
			ph[i][1] = phAb*Math.sin(phPh);
		}
		ph[0][0] = 0.0; ph[0][1] = 0.0;
		ph[nnx-1][0] = 0.0; ph[nnx-1][1] = 0.0;
		normalize(ph);
	}

	function setHarmonicPotential(xPos, vvextAtXMax) {
		const nnx=g_NNx;

		const k0 = vvextAtXMax/(nnx*g_dx*nnx*g_dx/4.0);
		for (let i=0; i<nnx; i++) {
			const x = i*g_dx;
			g_vvext[i] = k0*(x-xPos)*(x-xPos);
		}
	}

	function setJelliumPotential(xPos, r0, charge) { // r0:jellium radius
		const nnx=g_NNx;

		for (let i=0; i<nnx; i++) {
			const x = i*g_dx;
			const r = Math.sqrt((x-xPos)*(x-xPos));
			g_vvext[i]= (r>a) ? -charge/r : -(charge/r0)*(1.5-0.5*(r*r/(r0*r0)));
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc, lossSW, qmdSW ) {
		// lossSW 0-lossOFF 1-lossON 2-loss+trans 3-loss+trans,t-stop

		g_iterCount += 1;
		if (lossSW<3) {
			for (let i=0; i<nCalc; i++) {
				evolveTimeStep(qmdSW,g_timeStep);
			}
		}

		for (let ie=0; ie<g_numberOfState; ie++) { // calc energy
			const ib = g_psis[ie][0];
			g_energy[ie] = kineticEnergy(g_psi[ib])+potentialEnergy(g_psi[ib],g_vv);
		}
		sortState(g_numberOfState);

		if (lossSW==1 || lossSW==2 || lossSW==3) {
			setLoss(g_dampingFactor);
			GramSchmidt(g_numberOfState);
			sortState(g_numberOfState);
			if (lossSW==2 || lossSW==3) {
				setOcc(g_numberOfState,g_numberOfElectron);
			}
		}
	}

	// --- evolve timeStep

	function evolveTimeStep(qmdSW,dt) {

		g_sysTime += 0.5*dt;

		for (let ib=0; ib<g_numberOfOrbit; ib++) {
			kxStep(g_psi[ib],0.5*dt);
		}

		setElectronDensity(g_numberOfState);
		setEffectivePotential();
		for (let ib=0; ib<g_numberOfOrbit; ib++) {
			phaseStep(g_psi[ib],g_vv,dt);
		}

		g_sysTime += 0.5*dt;

		for (let ib=0; ib<g_numberOfOrbit; ib++) {
			kxStep(g_psi[ib],0.5*dt);
		}

		if (qmdSW==1) moveNuc(dt);
	}

	// --- kx step

	function kxStep(ph, deltat) { // ph[][]
		const nnx=g_NNx;

		const a = 4.0*g_dx*g_dx/deltat;
		const bbRe = 2.0;
		const bbIm = a;
		const aaRe = -2.0;
		const aaIm = a;
		const aaAb = aaRe*aaRe+aaIm*aaIm;

		for (let i=1; i<nnx-1; i++) {
			g_bRe[i] = bbRe*ph[i][0]-bbIm*ph[i][1] - ph[i+1][0] - ph[i-1][0];
			g_bIm[i] = bbRe*ph[i][1]+bbIm*ph[i][0] - ph[i+1][1] - ph[i-1][1];
		}

		g_uRe[1] = aaRe/aaAb;
		g_uIm[1] = -aaIm/aaAb;
		ph[1][0] = g_bRe[1]*g_uRe[1] - g_bIm[1]*g_uIm[1];
		ph[1][1] = g_bIm[1]*g_uRe[1] + g_bRe[1]*g_uIm[1];

		for (let i=2; i<nnx-1; i++) {
			const auAb = (aaRe-g_uRe[i-1])*(aaRe-g_uRe[i-1])+(aaIm-g_uIm[i-1])*(aaIm-g_uIm[i-1]);
			g_uRe[i] = (aaRe-g_uRe[i-1])/auAb;
			g_uIm[i] = -(aaIm-g_uIm[i-1])/auAb;
			ph[i][0] = (g_bRe[i]-ph[i-1][0])*g_uRe[i] - (g_bIm[i]-ph[i-1][1])*g_uIm[i];
			ph[i][1] = (g_bRe[i]-ph[i-1][0])*g_uIm[i] + (g_bIm[i]-ph[i-1][1])*g_uRe[i];
		}

		for (let i=nnx-3; i>=1; i--) {
			ph[i][0] -= ph[i+1][0]*g_uRe[i] - ph[i+1][1]*g_uIm[i];
			ph[i][1] -= ph[i+1][0]*g_uIm[i] + ph[i+1][1]*g_uRe[i];
		}
	}

	// --- phase step

	function phaseStep(ph, vv, deltat) { // ph[][], vv[]
		const nnx=g_NNx;

		for (let i=1; i<nnx-1; i++) {
			const th = deltat*vv[i];
			const cs = Math.cos(th), sn = Math.sin(th);
			const phr = ph[i][0];
			const phi = ph[i][1];
			ph[i][0] = cs*phr+sn*phi;
			ph[i][1] = cs*phi-sn*phr;
		}
	}

	// --- set electron density

	function setElectronDensity(istateMax) {
		const nnx=g_NNx;

		g_rho[0] = 0.0; g_rho[nnx-1] = 0.0;
		for (let i=1; i<nnx-1; i++) {
			g_rho[i] = 0.0;
			for (let ie=0; ie<istateMax; ie++) {
				const ib = g_psis[ie][0];
				const psibi = g_psi[ib][i];
				g_rho[i] += g_occ[ie]*(psibi[0]*psibi[0]+psibi[1]*psibi[1])/g_lylz;
			}
		}
	}

	// --- set effective potential

	function setEffectivePotential() {
		const nnx=g_NNx;

		poisson(100);
		setVxc();
		for (let i=0; i<nnx; i++) {
			g_vv[i] = g_vvext[i]+g_vvh[i]+g_vvx[i]+g_vvc[i];
		}
	}

	function  poisson(iterMax) {
		const nnx=g_NNx, h2=g_dx*g_dx, w=(1/2)*1.8; // 1.8 = SOR_omega

		for (let iter=1; iter<iterMax; iter++) {
			for (let i=1; i<nnx-1; i++) {
				g_vvh[i] = g_vvh[i]+w*(g_vvh[i+1]+g_vvh[i-1]-2.0*g_vvh[i]+h2*g_rho[i] );
			}
		}
	}

	// LDA :  J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981)
	function setVxc() {
		const nnx=g_NNx;

		const c1 = -0.984745022;
		for (let i=1; i<nnx-1; i++) {
			const rh = g_rho[i];
			const rh3 = Math.pow(rh,0.33333333);
			g_vvx[i] = c1*rh3;

			const rs = 0.6204/(rh3+1.0e-20);
			if (rs>=1.0) {
				const sqrtrs = Math.sqrt(rs);
				const ec = -0.1423/(1.0+1.0529*sqrtrs+0.3334*rs);
				g_vvc[i] = ec*(1.0+1.22838*sqrtrs+0.4445*rs)/(1.0+1.0529*sqrtrs+0.3334*rs);
			} else {
				g_vvc[i] = -0.05837-0.0084*rs +(0.0311+0.00133*rs)*Math.log(rs);
			}
		}
	}

	function eeCorrelation(rh) {
		const r = 0.6204/(Math.pow(rh,0.33333333)+1.0e-20);
		let ec;
		if (r>=1.0) {
			ec = -0.1423/(1.0+1.0529*Math.sqrt(r)+0.3334*r);
		} else {
			ec = -0.0480-0.0116*r+(0.0311+0.0020*r)*Math.log(r);
		}
		return ec;
	}

	// --- sort state

	function sortState(maxState){
		let w,iw;
		for (let ist=0; ist<maxState-1; ist++) {
			if (g_energy[ist]>g_energy[ist+1]+0.0001) {
				iw = g_psis[ist][0]; g_psis[ist][0]=g_psis[ist+1][0]; g_psis[ist+1][0]=iw;
				iw = g_psis[ist][1]; g_psis[ist][1]=g_psis[ist+1][1]; g_psis[ist+1][1]=iw;
				w = g_occ[ist]; g_occ[ist]=g_occ[ist+1]; g_occ[ist+1]=w;
				w = g_energy[ist]; g_energy[ist]=g_energy[ist+1]; g_energy[ist+1]=w;
			}
		}
	}

	// --- set loss

	function setLoss(damp) {
		for (let ib=0; ib<g_numberOfOrbit; ib++) {
			steepestDescent(g_psi[ib], g_vv, damp);
		}
	}

	function steepestDescent(ph, v, damp) { // ph[][], v[]
		const nnx=g_NNx, h2=2.0*g_dx*g_dx;
		const ee = kineticEnergy(ph) + potentialEnergy(ph,v);

		for (let i=1; i<nnx-1; i++) {
			g_wrk[i][0] = (2.0*ph[i][0]-ph[i+1][0]-ph[i-1][0])/h2+(v[i]-ee)*ph[i][0];
			g_wrk[i][1] = (2.0*ph[i][1]-ph[i+1][1]-ph[i-1][1])/h2+(v[i]-ee)*ph[i][0];
		}

		for (let i=1; i<nnx-1; i++) {
			ph[i][0] -= damp*g_wrk[i][0];
			ph[i][1] -= damp*g_wrk[i][1];
		}
	}

	// --- Gram-Schmidt

	function GramSchmidt(stateMax) {
		const nnx=g_NNx, dx=g_dx, psis=g_psis, psi=g_psi;

		normalize(psi[0]);

		for (let istate=1; istate<stateMax; istate++) {
			const ibstate = psis[istate][0]; // spacial orbit of istate state
			for (let ist=0; ist<istate; ist++) {
				const ibst = psis[ist][0]; // spacial orbit of ist state
				if (psis[ist][1]==psis[istate][1]) { // pararell spin
					let pRe = 0.0, pIm = 0.0;
					for (let i=1; i<nnx-1; i++) {
						pRe += (psi[ibst][i][0]*psi[ibstate][i][0] + psi[ibst][i][1]*psi[ibstate][i][1])*dx;
						pIm += (psi[ibst][i][0]*psi[ibstate][i][1] - psi[ibst][i][1]*psi[ibstate][i][0])*dx;
					}
					for (let i=1; i<nnx-1; i++) {
						psi[ibstate][i][0] -= (pRe*psi[ibst][i][0] - pIm*psi[ibst][i][1]);
						psi[ibstate][i][1] -= (pRe*psi[ibst][i][1] + pIm*psi[ibst][i][0]);
					}
				}
			}
			normalize(psi[ibstate]);
		}
	}

	// --- set occupation

	function setOcc(maxState, nElectron) {

		let eUpper = g_energy[maxState-1];
		let eLower = g_energy[0];
		for (let i=0; i<maxState; i++) {
			if (g_energy[i]>eUpper) eUpper = g_energy[i];
			if (g_energy[i]<eLower) eLower = g_energy[i];
		}

		while (eUpper-eLower>1.0e-12) {
			const eFermi = (eUpper+eLower)/2.0;
			const ntrial = trialOcc(maxState, eFermi);
			if (ntrial<nElectron) {
				eLower = eFermi;
			} else {
				eUpper = eFermi;
			}
		}
		const eFermi = (eUpper+eLower)/2.0;

		for (let i=0; i<maxState; i++) {
			g_occ[i] = 1.0*FermiDirac(g_energy[i], eFermi);
			if (g_occ[i]<0.0001) g_occ[i] = 0.0;
			if (1.0-g_occ[i]<0.0001) g_occ[i] = 1.0;
		}
	}

	function trialOcc(maxState, eFermi) {
		let s = 0.0;
		for (let i=0; i<maxState; i++) {
			s += 1.0*FermiDirac(g_energy[i], eFermi);
		}
		return s;
	}

	function FermiDirac(ee, ef) {
		const et = 0.005;
		return ( 1.0/(Math.exp((ee-ef)/et)+1.0) );
	}

	function levelWidth() { // no use

		let kT = broadening;
		if (g_iterCount<30) {
			kT=0.1;
		} else {
			kT = 0.1-0.002*(g_iterCount-29);
			if (kT<broadening) kT = broadening;
		}
		return kT;
	}


	// --- utility

	function norm(ph) { // ph[][]
		const nnx=g_NNx;
		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			s += (ph[i][0]*ph[i][0]+ph[i][1]*ph[i][1]);
		}
		return s*g_dx;
	}

	function normalize(ph) { // ph[][]
		const nnx=g_NNx;
		const a = Math.sqrt(norm(ph));

		for (let i=1; i<nnx-1; i++) {
			ph[i][0] = ph[i][0]/a;
			ph[i][1] = ph[i][1]/a;
		}
	}

	function kineticEnergy(ph) { // ph[][]
		const nnx=g_NNx, dx=g_dx;

		let s=0.0;
		for (let i=1; i<nnx-1; i++) {
			const hphRe = (2.0*ph[i][0]-ph[i+1][0]-ph[i-1][0])/(2.0*dx*dx);
			const hphIm = (2.0*ph[i][1]-ph[i+1][1]-ph[i-1][1])/(2.0*dx*dx);
			s += (ph[i][0]*hphRe + ph[i][1]*hphIm)*dx;
		}
		return s;
	}

	function momentum(ph) { // ph[][]
		const nnx=g_NNx, dx=g_dx;

		let s=0.0;
		for (let i=1; i<nnx-1; i++) {
			const pphRe = (ph[i+1][1]-ph[i-1][1])/(2.0*dx);
			const pphIm = (-ph[i+1][0]+ph[i-1][0])/(2.0*dx);
			s += (ph[i][0]*pphRe + ph[i][1]*pphIm)*dx;
		}
		return s;
	}

	function potentialEnergy(ph, v) { // ph[][], v[]
		const nnx=g_NNx, dx=g_dx;

		let s = 0.0;
		for (let i=1; i<nnx-1; i++) {
			s += v[i]*(ph[i][0]*ph[i][0] + ph[i][1]*ph[i][1])*dx;
		}
		return s;
	}

	function innerProduct(f, g) { // f[][], g[][]
		const nnx=g_NNx, dx=g_dx;

		let pRe=0.0, pIm=0.0;
		for (let i=1; i<nnx-1; i++) {
			pRe += (f[i][0]*g[i][0] + f[i][1]*g[i][1])*dx;
			pIm += (f[i][0]*g[i][1] - f[i][1]*g[i][0])*dx;
		}
		return Math.sqrt(pRe*pRe+pIm*pIm);
	}

	function meanPosX(ph) { // ph[][]
		const nnx=g_NNx,dx=g_dx;

		var s=0.0;
		for (let i=1; i<nnx-1; i++) {
			s += i*dx*(ph[i][0]*ph[i][0] + ph[i][1]*ph[i][1])*dx;
		}
		return (s - 8.0);
	}


	// ------------------------------------ move nuclei -----------
	//
	//  length : 1 au = 5.29177e-11 m
	//  time   : 1 au = 2.41888e-17 s
	//  energy : 1 au = 4.38975e-18 J (= 27.2114 eV)
	//
	//  int g_nucMax  max number of nucleus (i = 0...g_nucMax-1), g_nucMax must less then 8
	//  double g_nucMass[i] i-th nuclear mass in SI(kg)
	//  double g_nucCharge[i] i-th nuclear charge in au (electron charge = -1)
	//  double g_nucxx[i] i-th nuclear position in au
	//  double g_nucvx[i] i-th nuclear velocity in SI(m/s)
	//  double g_nucfx[i] i-th nuclear force in SI(N)
	//

	function setInitNuc(iNuc, nuclearMass, nuclearCharge, nuclearxx, nuclearvx) {
		g_nucMass[iNuc] = nuclearMass*1.67e-27; // kg
		g_nucCharge[iNuc] = nuclearCharge;
		g_nucxx[iNuc] = nuclearxx;
		g_nucvx[iNuc] = nuclearvx;
		g_nucfx[iNuc] = 0.0;
	}

	function moveNuc(deltat) {
		const nNuc=g_nNuc;
		const dtNuc = deltat*g_auTime; // au to SI unit

		for (let i=0; i<nNuc; i++) {
			g_nucvx[i] += 0.5*dtNuc*g_nucfx[i]/g_nucMass[i]; // in SI
			g_nucxx[i] += (g_nucvx[i]*dtNuc)/g_auLength; // in au
		}
		setNucForce();
		for (let i=0; i<nNuc; i++) {
			g_nucvx[i] += 0.5*dtNuc*g_nucfx[i]/g_nucMass[i]; // in SI
		}
		setNucleiPotential();
	}

	function setNucForce() {
		const nNuc=g_nucMax;

		for (let i=0; i<nNuc; i++) {
			g_nucfx[i] = (4.38975e-18/5.29177e-11)*electronForceAtNuc(g_nucxx[i]);
		}
		for (let i=0; i<nNuc; i++) {
			for (let j=i+1; j<nNuc; j++) {
				const f = (4.38975e-18/5.29177e-11)*forceNN(i,j);
				g_nucfx[i] += f;
				g_nucfx[j] -= f;
			}
		}
	}

	function electronForceAtNuc(xPos) {
		const nnx=g_NNx, dx=g_dx

		let s=0.0;
		for (let i=1; i<nnx-1; i++) {
			const x = i*dx;
			const r2 = (x-xPos)*(x-xPos);
			const r = Math.sqrt(r2);
			if (r2>0.1) {
				s += (x-xPos)/(r2*r)*g_rho[i]*g_lylz*dx;
			}
		}
		return s;
	}

	function forceNN(i, j) {
		const x1 = g_nucxx[i], x2 = g_nucxx[j];
		const r = Math.sqrt((x1-x2)*(x1-x2));
		return ( (x1-x2)*g_nucCharge[i]*g_nucCharge[j]/(r*r*r) );
	}

	function setNucleiPotential() {
		const nnx=g_NNx, nNuc=g_nucMax;

		for (let i=0; i<nnx; i++) {
			g_vvext[i] = 0.0;
		}

		for (let iNuc=0; iNuc<nNuc; iNuc++) {
			addNucPotential(iNuc);
		}
		addHillPotential();
	}

	function addNucPotential(iNuc) {
		const nnx=g_NNx, dx=g_dx;

		const a = g_jelliumRadius;
		const qq = g_nucCharge[iNuc]; // charge
		const xPos = g_nucxx[iNuc];

		for (let i=0; i<nnx; i++) {
			const x = i*dx;
			const r = Math.sqrt((x-xPos)*(x-xPos));
			if (r>a) {
				g_vvext[i] += -qq/r;
			} else {
				g_vvext[i] += -(qq/a)*(1.5-0.5*(r*r/(a*a)));
			}
		}
	}

	function addHillPotential() {
		const nnx=g_NNx, dx=g_dx;
		const hillLen = nnx*dx*0.2;

		for (let i=0; i<nnx; i++) {
			const x = i*dx;
			if (x<hillLen || x>NNx*dx-hillLen) {
				g_vvext[i] += 1.0;
			}
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( nCalc, lossSW, qmdSW )

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_dx, g_timeStep, g_numberOfState ]; },
		getNow:			function() { return [ g_iterCount, g_sysTime ]; },
		getOccupation:	function(ist) { return g_occ[ist]; },
		getStEnergy:	function(ist) { return g_energy[ist]; },
		getState:		function(ist) { return [ g_psis[ist][0], g_psis[ist][1] ]; }, // [ orbit, spin ]
		getPsi:			function(orbit,i) { return g_psi[orbit][i]; }, // [ RePsi, ImPsi ]
		getDensity:		function(i) { return g_rho[i]; },
		getVext:		function(i) { return g_vvext[i]; },
		getVeff:		function(i) { return g_vv[i]; },
		getVh:			function(i) { return g_vvh[i]; },
		getVx:			function(i) { return g_vvx[i]; },
		getVc:			function(i) { return g_vvc[i]; },
	};

})(); // ====================  electronsTDKS1D end  ====================


const js098 = (function(){ // ====================  js Module  ====================

	const theModule = electronsTDKS1D;
	const themeStr = [ 'Li', 'Na','K ', 'Rb','Cs' ];
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;

	let v_theme = 0;	// 0:(up,up) 1:(up,down) 2:(up,up,down) 3:(up,up,up,up)
	let v_nCalc = 2;	// n-times evolve time step
	let v_lossSW = 0;	// 0-lossOFF 1-lossON 2-loss+trans 3-loss+trans,t-stop
	let v_qmdSW = 0;	// QMD(nuclear motion) 0:off 1:on

	let p_NNx, p_dx, p_timeStep, p_numberOfState; // = theModule.getSysParam();
	let p_NNy, p_NNz;

  let iterCount, sysTime;
  let occList = [];
  let stEnergyList = [];
	let spinList =[];
  let densityArray = [];
	let vextArray = [];

	let dispMode = 3;
	let dispState = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let inStepFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1;
  let countStamp;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme );
			[ p_NNx, p_dx, p_timeStep, p_numberOfState ] = theModule.getSysParam();
			p_NNy = p_NNx;
			p_NNz = p_NNx;
			changeStateSelectorOptions( p_numberOfState );
			v_lossSW = 0;
			document.getElementById("slct_loss").selectedIndex = 0;
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
			getFieldFlag = true;
      fieldKind = 1;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc, v_lossSW, v_qmdSW );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc, v_lossSW, v_qmdSW );
			inStepFlag = true;
		}

		draw( ctx, dispMode );

    if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    for (let ist=0; ist<p_numberOfState; ist++) {
      occList[ist] = theModule.getOccupation(ist);
      stEnergyList[ist] = theModule.getStEnergy(ist);
      spinList[ist] = theModule.getState(ist)[1];
    }

    if (fieldKind==1) {
      countStamp = iterCount;
      for (let i=0; i<p_NNx; i++) {
        densityArray[i] = theModule.getDensity(i);
				vextArray[i] = theModule.getVext(i);
      }
    }
  }


	// --------------------  draw  --------------------

	const gColor = { orb:"#dddd00", dens:"#dd88dd",
					Vext:"#00dd00", Veff:"#0088ff", Vh:"#4444ff", Vxc:"#8800ff", Vx:"#aa00ff",Vc:"#8888ff" };

	function draw( ctx, dispMode ) {

		const xp = 30, yp = 20, xtabp = 320;

		let auLength, auTime, auEnergy, au2eV;
		[ auLength, auTime, auEnergy, au2eV ] = theModule.getAUinSI();
		[ iterCount, sysTime ] = theModule.getNow();

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if (dispMode==0) {
			dispText( "state table" );
			drawStateTable( ctx );
		} else if (dispMode==1) {
			dispText( "density(x) + Vext(x)" );
			drawLines3D( ctx, 0 );

		} else if (dispMode==2) {
			dispText( "state(x) + Vext(x)" );
			drawLines3D( ctx, 1 );
		} else if (dispMode==3) {
			dispText( "occupied states + Vext(x)" );
			drawLines3D( ctx, 2 );

		} else if (dispMode==4) {
			dispText( "potential + density" );
			drawLines3D( ctx, 3 );

		}

		// caption
		ctx.fillStyle = "#888888";
		ctx.fillText(`box = ${(p_NNx*p_dx)} (au)`, 20, yCanvasSize-30);
		//ctx.fillText("Z ="+p_valenceZ+",  ne ="+(p_valenceZ+v_dzNu), 260, yCanvasSize-30);
		ctx.fillText(`iteration = ${iterCount}`, 20, yCanvasSize-10);
		ctx.fillText(`time = ${sysTime.toFixed(3)} (au)`, 260, yCanvasSize-10);

		function dispText( str ) {
			ctx.fillStyle = "#888888";
			ctx.fillText( str, 20, yCanvasSize-50 );
		}
	}


	function drawLines3D( ctx, mode ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy;
		const vmag = 5.0, dmag = 10000.0, pmag = 20.0;

		const densVextFunc = function(i) { return vmag*theModule.getVext(i) + dmag*theModule.getDensity(i); };
		const vxcFunc = function(i) { return (theModule.getVx(i)+theModule.getVc(i)); };

		g3d.set3DRotateXY(0.0);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow

		if ( mode==0 ) {
			yPlot( densVextFunc, 1.0, gColor.dens );
			yPlot( theModule.getVext, vmag, gColor.Vext );
			ctx.fillStyle = gColor.dens;
			ctx.fillText(`density(x)`, 300, 20);
			ctx.fillStyle = gColor.Vext;
			ctx.fillText(`Vext(x)`, 400, 20);

		} else if ( mode==1 ) {
			yPlot( theModule.getVext, vmag, gColor.Vext );
			const state = dispState;
			const energy = theModule.getStEnergy( state );
			const occ = theModule.getOccupation( state );
			let orb, spin;
			[ orb, spin ] = theModule.getState( state );
			orbitPlot( orb, pmag, vmag*energy, occ );
			const spinStr = ((spin==1) ? "up" : "down");
			ctx.fillStyle = "#888888";
			ctx.fillText(`|${state}>   energy:${energy.toFixed(6)},  occ:${occ.toFixed(4)},  spin:${spinStr}`, 20, 20);
			ctx.fillStyle = gColor.Vext;
			ctx.fillText(`Vext(x)`, 400, 20);

		}  else if ( mode==2 ) {
			yPlot( theModule.getVext, vmag, gColor.Vext );
			let ypp = 16;
			for (let ist=0; ist<p_numberOfState; ist++) {
				const energy = theModule.getStEnergy( ist );
				const occ = theModule.getOccupation( ist );
				if ( occ>0.1 ) {
					let orb, spin;
					[ orb, spin ] = theModule.getState( ist );
					const color = `hsl(${(ist*30)},${Math.floor(occ*80.0)+20}%,50%)`
					orbitPlot( orb, pmag, vmag*energy, occ, color );
					ctx.font = "12px 'sans-serif'";
					ctx.fillStyle = color;
					ctx.fillText(`|${ist}>`, 20, ypp);
					ctx.fillStyle = "#888888";
					ctx.fillText(
						`energy:${energy.toFixed(6)},   occ:${occ.toFixed(4)},   spin:${((spin==1) ? "up" : "down")}`,
						50, ypp);
					ctx.font = "16px 'sans-serif'";
					ypp +=15;
				}
			}
			ctx.fillStyle = gColor.Vext;
			ctx.fillText(`Vext(x)`, 400, 20);

		} else if ( mode==3 ) {
			yPlot( densVextFunc, 1.0, gColor.dens );
			yPlot( theModule.getVext, vmag, gColor.Vext );
			yPlot( theModule.getVeff, vmag, gColor.Veff );
			yPlot( theModule.getVh, vmag*10, gColor.Vh );
			yPlot( vxcFunc, vmag*10, gColor.Vxc );
			ctx.fillStyle = gColor.dens;
			ctx.fillText(`density(x)`, 20, 20);
			ctx.fillStyle = gColor.Vext;
			ctx.fillText(`Vext(x)`, 120, 20);
			ctx.fillStyle = gColor.Veff;
			ctx.fillText(`Veff(x)`, 200, 20);
			ctx.fillStyle = gColor.Vh;
			ctx.fillText(`Vh(x) x10`, 280, 20);
			ctx.fillStyle = gColor.Vxc;
			ctx.fillText(`Vxc(x) x10`, 380, 20);
		}

		g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow
		/*
		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
		*/

		function yPlot( yFunc, mag, color ) {
			let yi, yip;
			for (let i=0; i<nnx-1; i++) {
				yi = mag*yFunc( i );
				const x = i*g_dx, y = g3d.cy0-yi, z = g3d.cz0;
				yip = mag*yFunc( i+1 );
				const x1 = (i+1)*g_dx, y1 = g3d.cy0-yip, z1 = g3d.cz0;
				g3d.drawRotatedLine(ctx, x, y, z, x1, y1, z1, color, sc, xp, yp );
			}
		}

		function orbitPlot( orb, mag, yConst, occ, color ) {
			const degree = 180.0/Math.PI, colorUdefined = ( color==undefined ) ? true : false;
			let yi, zi, yip, zip;
			for (let i=0; i<nnx-1; i++) {
				[ yi, zi ] = yzFunc( orb, i, mag );
				const x = i*g_dx, y = g3d.cy0-yi-yConst, z = g3d.cz0+zi;
				[ yip, zip ] = yzFunc( orb, i+1, mag );
				const x1 = (i+1)*g_dx, y1 = g3d.cy0-yip-yConst, z1 = g3d.cz0+zip;
				const amp = Math.sqrt(yi*yi+zi*zi + yip*yip+zip*zip);
				let col;
				if ( colorUdefined ) {
					const th = Math.atan2(zi+zip,yi+yip)*degree+180.0;
					col = `hsl(${th},${Math.floor(occ*80.0)+20}%,50%)`;
				} else {
					col = color;
				}
				const colr = (amp>0.5) ? col : '#444444';
				g3d.drawRotatedLine(ctx, x, y, z, x1, y1, z1, colr, sc, xp, yp );
			}

			function yzFunc( orb, i, mag ) {
				let re, im;
				[ re, im ] = theModule.getPsi(orb,i);
				return [ mag*re, mag*im ];
			}
		}

	};

	function drawStateTable( ctx ) {
		const nst = p_numberOfState;
		const px = 15, py = 20;

		// clear canvas
		ctx.fillStyle = "#888888";
		ctx.fillRect( 5, 5, xCanvasSize-10, yCanvasSize-80 );

		ctx.fillStyle = "rgb(0,0,0)";
		ctx.fillText("State Table", px+50, py+20);
		ctx.fillText("state", px+30, py+60);
		ctx.fillText("state E (au)", px+90, py+60);
		ctx.fillText("occupation", px+190, py+60);
		ctx.fillText("orbNo.", px+290, py+60);
		ctx.fillText("spin", px+360, py+60);

		for (let ist=0; ist<nst; ist++) {
			const energy = theModule.getStEnergy( ist );
			const occ = theModule.getOccupation( ist );
			let orb, spin;
			[ orb, spin ] = theModule.getState( ist );

			const ppy = py+85+(nst-ist-1)*20;
			ctx.fillStyle = `hsl(${240-occ*240},100%,50%)`;
			ctx.fillText(`| ${ist} >`, px+30, ppy);
			ctx.fillText(`${energy.toFixed(6)}`, px+90, ppy);
			ctx.fillText(`${occ.toFixed(6)}`, px+190, ppy);
			ctx.fillText(`${orb}`, px+290, ppy);
			ctx.fillText(((spin==1) ? "up" : "down"), px+360, ppy);
		}
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
		document.getElementById("slct_orbit").disabled = true;
	}

	function changeStateSelectorOptions( nSelect ) {
		const element = document.getElementById( "slct_orbit" );
		element.length = 0; // delete all options
		for (let i=0; i<nSelect; i++) {
			const option = document.createElement("option");
 			option.text = i;
 			element.appendChild(option);
		}
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = 1;
	}

	function setLossSW() {  // select lossSW
		v_lossSW = 0 + document.getElementById("slct_loss").selectedIndex;
	}

	function setDispMode() {  // select dispMode
		const dom = document.getElementById("slct_orbit");
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
		dom.disabled = ( dispMode==2 ) ? false : true;
	}

	function setOrbit() {  // select dispOrbit
		dispState = 0 + document.getElementById("slct_orbit").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(-10,-10);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ iterCount, sysTime ];
  }

  function pygetOrbitData() {
    return [ occList, stEnergyList, spinList ];
  }

  function pygetFieldData() {
		fieldKind = 0;
    return [ countStamp, vextArray, densityArray ];
  }


	// --------------------  public  --------------------

	return {
		main:				main,				// main()

		reset:				reset,				// reset()
		pause:				pause,				// pause()
		step:				step,				// step()

		setTheme:			setTheme,			// setTheme()
		setLossSW:			setLossSW,			// setLossSW()

		setDispMode:		setDispMode,		// setDispMode()
		setOrbit:			setOrbit,			// setOrbit()
		viewHome:			viewHome,			// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ iterCount, sysTime ]
    pygetOrbitData: pygetOrbitData, // pygetOrbitData() : return [ occList, stEnergyList, spinList ]
    pygetFieldData: pygetFieldData, // ygetFieldData() : return [ countStamp, vextArray, densityArray ]
	};

})(); // ====================  js098 end  ====================


const js = js098;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js098]  electrons - time dependent Kohn-Sham (TDKS1D)</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option selected>2-electrons spin-up,up</option><option>2-electrons spin-up,down</option>
<option>3-electrons spin-up,up,down</option><option>4-electrons spin-up,up,up,up</option>
</select>
    <span style="margin-right: 20px;"></span>
in parabolic Vext(x) = k x^2
<br>

<label>loss:</label>
<select id="slct_loss" onChange="js.setLossSW()">
<option>off</option><option>on</option><option>on+trans</option>
<option>on+trans,time stop</option>
</select>
    <span style="margin-right: 160px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>state table</option><option>density(x) + Vext(x)</option>
<option>state(x) + Vext(x)</option><option selected>occupied states + Vext(x)</option>
<option>potentials + density </option>
</select>
    <span style="margin-right: 20px;"></span>
<label>orbit:</label>
<select id="slct_orbit" onChange="js.setOrbit()">
<option selected>0</option><option>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option><option>7</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# get data and print
for i in range(10):
  [ iterCount, sysTime ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2},  count = {iterCount:>4},  system time = {sysTime:>8.2f} (au)')
  time.sleep(3)

# get orbit occupation and energy , and print
[ occList, energyList, spinList ] = eval_js( 'js.pygetOrbitData()' )
print("")
print( "orbit list :", [i for i in range(len(occList))] )
print( "orbit occupation list :", occList )
print( "orbit energy list :", energyList )
print( "orbit spin list :", ['up' if spn>0 else 'down' for spn in spinList] )

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, change theme, dispMode

import time

themeList = [
  '0: 2-electrons spin-up,up', '1: 2-electrons spin-up,down',
  '2: 3-electrons spin-up,up,down', '3: 4-electrons spin-up,up,up,up' ]
dispModeList = [
  '0: state table', '1: density(x) + Vext(x)', '2: state(x) + Vext(x)',
  '3: occupied states + Vext(x)', '4: potentials + density' ]

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# set theme
theme = 2 # '2: 3-electrons spin-up,up,down'
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- set theme :", themeList[theme], " --")
time.sleep(3)

# chabge disp mode and get data and print
for dispMode in range(len(dispModeList)):
  # change dispMode
  print(f"  -- dispMode: {dispModeList[dispMode]} --")
  eval_js( 'js.pysetDispMode({})'.format(dispMode) )

  # get data and print
  [ iterCount, totalEnergy ] = eval_js( 'js.pygetData({})'.format(dispMode) )
  print(f'\t  count = {iterCount:>4},  total energy = {totalEnergy:>10.6f} (au)')
  time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get electron density --> density, vext

import time
import numpy as np

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# get data and print
for i in range(10):
  [ iterCount, sysTime ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2},  count = {iterCount:>4},  system time = {sysTime:>8.2f} (au)')
  time.sleep(3)

# get densityArray data
[ countStamp, vextList, densityList ] = eval_js( 'js.pygetFieldData()' )
print("-- get vext, dens list --")

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# convert to nampy array
print("-- convert to numpy array Dens --")
Vext = np.array(vextList)
Dens = np.array(densityList)
print(f"-- count = {countStamp},  Vext shape :{Vext.shape} , Dens shape :{Dens.shape}  --")


In [ ]:
# save particles data

import numpy as np

packed_data = np.array([ vextList, densityList ])

# print data shape
print(f"--  packed_data array shape :{packed_data.shape} --")

# save
print("-- save data as 'js098_data.npy' --")
np.save( 'js098_data.npy', packed_data )


# The saved file can be viewed by clicking on the folder icon on the left edge of the notebook.
# You must download this file to save it permanently.
# Select Download from the '...'.T menu of the file to download it.

In [ ]:
# electron density: Dens(numpy 3d array) --> scatter plot 3D

import numpy as np
import matplotlib.pyplot as plt

loaded_data = np.load('js098_data.npy')

# unpack np data
Vext = loaded_data[0]
Dens = loaded_data[1]

plt.plot(Dens*10000, 'r-')
plt.plot(Vext, 'g-')
plt.xlabel("x")
plt.title("electron density + Vext(x)")
plt.show()